# **Forward and Backward Passes**

In this notebook we'll be looking at setting up the forward and backward passes for the fully connected model architecture.

## **Colab Setup**
The setup structure for this will depend on the environment. I'm assuming a Google Colab environment in this case, which will require the following setup from the Github repo, and assumes that the repo has already been cloned into Google Drive:

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

%cd gdrive/My Drive/git_folder/DL_From_Foundations
! git pull

In [2]:
%load_ext autoreload
%autoreload 2

%matplotlib inline



---



## **Normalize Data**

In [6]:
#export
from exp.nb_01 import *
from keras.datasets import mnist

def get_data():
    # Load the data into the train and validation sets
    (x_train, y_train), (x_valid, y_valid) = mnist.load_data()

    # Map the sets to tensors
    x_train,y_train,x_valid,y_valid = map(tensor, (x_train,y_train,x_valid,y_valid))

    # Flatten the 28 * 28 to match the course
    x_train = torch.flatten(x_train, 1)
    x_valid = torch.flatten(x_valid, 1)

    return x_train, x_valid, y_train, y_valid

def normalize(x, m, s): return (x-m)/s

In [8]:
x_train,y_train,x_valid,y_valid = get_data()

train_mean,train_std = x_train.float().mean(),x_train.float().std()
train_mean,train_std

(tensor(33.3184), tensor(78.5675))

In [9]:
x_train = normalize(x_train, train_mean, train_std)

# NB: Use training, not validation mean for validation set
x_valid = normalize(x_valid, train_mean, train_std)

In [10]:
train_mean,train_std = x_train.mean(),x_train.std()
train_mean,train_std

(tensor(1.8892e-08), tensor(1.))

Now we're talking!



---



## **Foundations (v1.0)**